# FinTech Vendor Scorecard Development

This notebook implements the FinTech Vendor Scorecard for Micro-Lending.
It combines entities extracted by the NER model with Telegram post metadata
(views, timestamps, channel/vendor name) to create a rich profile for each vendor.
It then calculates key performance metrics and a weighted "Lending Score" for each.

This notebook is a core component of the FinTech E-commerce Data Extractor project,
specifically designed to fulfill the business objective of identifying promising
vendors for micro-lending opportunities within the EthioMart ecosystem.

**Project Goal:** To leverage unstructured Telegram e-commerce data to provide
data-driven insights for financial decision-making, particularly micro-lending.

**Business Objective:** Enable EthioMart to make informed micro-lending decisions
by quantifying vendor performance and potential based on their digital footprint
and engagement on Telegram channels. This aims to reduce risk and optimize lending
to active and successful small businesses.

**Methodology:**
The methodology integrates several stages of data processing and analysis:
1. **Data Integration:** Utilizes preprocessed Telegram message data, incorporating
critical metadata such as message views, posting message_dates, and channel information.
2. **Entity Extraction:** Employs a fine-tuned Named Entity Recognition (NER) model
to accurately extract crucial business entities (Product, Price, Location, Contact Info)
from the free-form Amharic text of Telegram posts.
3. **Vendor Analytics Engine Development:** Constructs a robust analytics engine
to compute key performance indicators for each vendor, transforming raw data
and extracted entities into quantifiable metrics.
4. **Lending Score Formulation:** Develops a composite "Lending Score" by normalizing
and weighting selected performance indicators, providing a single, interpretable
metric to rank vendors.

**Implementation Details:**
This notebook orchestrates the following implementation steps:
- Loading of preprocessed Telegram data and the custom-trained NER model (from a Hugging Face repository or local path).
- Performing high-throughput NER inference across all available messages to enrich vendor profiles with extracted product and pricing information.
- Calculating vendor-specific metrics including:
  - **Posting Frequency:** Measuring the regularity and volume of a vendor's posts.
  - **Average Views per Post:** Quantifying the reach and engagement of a vendor's content.
  - **Average Price Point:** Analyzing the typical pricing of products offered, indicating market segment.
  - **Top Performing Post Analysis:** Identifying the most successful posts and their associated product-price pairs.
- Normalizing these metrics and applying predefined weights to compute the final "Lending Score".
- Presenting a comprehensive summary table of the calculated metrics and Lending Scores,
sorted for immediate business insights, along with detailed interpretations of the results.

## 1. Setup and Library Installation

This section installs the necessary Python libraries for data manipulation, natural language processing (specifically Hugging Face's transformers and scikit-learn), and general utilities.


In [14]:
!pip install pandas numpy transformers scikit-learn -q

In [15]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
from datetime import time, timedelta
import re
from collections import defaultdict
import logging

# Set up logging for informative messages
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# logger = logging.getLogger(__name__)

# Import Hugging Face components for loading the fine-tuned model
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

print("Libraries imported successfully.")

Libraries imported successfully.


## 2. Define Paths and Configuration

Here, we define the file paths for our input data (preprocessed Telegram messages), the location of the fine-tuned NER model (either locally or on the Hugging Face Hub), and the desired output path for the final vendor scorecard CSV file. This section ensures that the notebook can locate the required resources and save its results correctly.



In [16]:
# Assuming the notebook is in 'notebooks/' and project root is its parent's parent
PROJECT_ROOT = Path('/content') # Assuming your project root is /content in Colab
sys.path.insert(0, str(PROJECT_ROOT))

# Paths to data and model
# Ensure preprocessed_telegram_data.csv has 'views', 'message_date', 'channel' columns
# (These should ideally be passed through during preprocessing or original merged data used)
PREPROCESSED_DATA_PATH = PROJECT_ROOT / 'data' / 'processed' / 'preprocessed_telegram_data.csv'

# -Load model from Hugging Face Hub ---
# Replace "YOUR_HF_USERNAME/your_ner_model_repo" with your actual Hugging Face model repository name.
# Example: "your-username/xlm-roberta-base-finetuned-amharic-ner"
# Ensure the model was pushed to the hub in Task 4.
FINE_TUNED_MODEL_NAME_OR_PATH = "michaWorku/amahric_ner_fine_tunning" # Default to local if not pushing to hub, or replace with your HF repo.
                                                  # e.g., "YOUR_HF_USERNAME/your_amharic_ner_model"
# For demonstration, we'll try to load locally first, then fallback to hub if specified.
# If you pushed your model to HF Hub, replace 'xlm-roberta-base' with your repo name.
# E.g., FINE_TUNED_MODEL_NAME_OR_PATH = "your_hf_username/your_finetuned_model_repo"
# If running locally without pushing to HF Hub, keep this as the path to your saved model directory.
if (PROJECT_ROOT / 'fine_tuned_ner_model').exists():
    FINE_TUNED_MODEL_NAME_OR_PATH = str(PROJECT_ROOT / 'fine_tuned_ner_model')
    print(f"Loading model from local path: {FINE_TUNED_MODEL_NAME_OR_PATH}")
else:
    # If the local path doesn't exist, assume it's a Hugging Face Hub model name
    # You MUST replace "YOUR_HF_USERNAME/your_ner_model_repo" with your actual model name on Hugging Face Hub
    # This is a placeholder and will likely fail if you haven't uploaded it.
    # FINE_TUNED_MODEL_NAME_OR_PATH = "xlm-roberta-base" # Fallback to base model for demo if local not found
    print(f"WARNING: Local fine-tuned model not found. Attempting to load '{FINE_TUNED_MODEL_NAME_OR_PATH}'. "
                   f"Please ensure your model is uploaded to Hugging Face Hub or exists locally.")


# Output for the vendor scorecard
VENDOR_SCORECARD_PATH = PROJECT_ROOT / 'data' / 'processed' / 'vendor_scorecard.csv'

# Create the directory for processed data if it doesn't exist
os.makedirs(PREPROCESSED_DATA_PATH.parent, exist_ok=True)


print(f"Project root: {PROJECT_ROOT}")
print(f"Preprocessed data path: {PREPROCESSED_DATA_PATH}")
print(f"Fine-tuned model loading from: {FINE_TUNED_MODEL_NAME_OR_PATH}")
print(f"Vendor scorecard output path: {VENDOR_SCORECARD_PATH}")

Project root: /content
Preprocessed data path: /content/data/processed/preprocessed_telegram_data.csv
Fine-tuned model loading from: michaWorku/amahric_ner_fine_tunning
Vendor scorecard output path: /content/data/processed/vendor_scorecard.csv


## 3. Load Data and Fine-tuned NER Model

This crucial step loads the preprocessed Telegram data into a pandas DataFrame. It includes checks to ensure the data file exists and contains the essential columns needed for the analysis ('preprocessed_text', 'views', 'message_date', 'channel_username'). The message_date column is converted to a datetime format, and rows with missing essential metadata are removed to maintain data integrity.

Subsequently, the fine-tuned Named Entity Recognition (NER) model and its corresponding tokenizer are loaded. The code attempts to load the model from the specified path (FINE_TUNED_MODEL_NAME_OR_PATH). If the local path does not exist, it assumes the path is a Hugging Face Hub model name and attempts to load it from there. An NER pipeline is then created for efficient entity extraction from the text data.

In [17]:
# Load preprocessed data (should ideally contain views, message_date, channel, and preprocessed_text)
if not PREPROCESSED_DATA_PATH.exists():
    print(f"Error: Preprocessed data not found at {PREPROCESSED_DATA_PATH}. Please ensure preprocessing output is available and includes metadata.")
    raise FileNotFoundError(f"Preprocessed data not found: {PREPROCESSED_DATA_PATH}")

try:
    df = pd.read_csv(PREPROCESSED_DATA_PATH)
    print(f"Loaded {len(df)} rows from {PREPROCESSED_DATA_PATH}")

    # Print head and columns for inspection
    print("\nDataFrame Head after initial load:")
    display(df.head())
    print("\nDataFrame Columns after initial load:")
    print(df.columns)


    # Basic check for essential columns for this analysis
    required_cols = ['preprocessed_text', 'views', 'message_date', 'channel_username']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        print(f"Error: Missing one or more required columns ({required_cols}) in the dataframe. This will prevent scorecard generation.")
        print("Please ensure src/data_preprocessing/text_processor.py::preprocess_dataframe passes through 'views', 'message_date', and 'channel' columns from the raw data.")
        raise ValueError(f"Essential metadata columns are missing in the input DataFrame: {missing}. Ensure preprocessing retains these fields.")

except Exception as e:
    print(f"Failed to load or valimessage_date data: {e}")
    raise

# Convert 'message_date' column to message_datetime objects
df['message_date'] = pd.to_datetime(df['message_date'], errors='coerce')
df.dropna(subset=['message_date', 'views', 'channel_username', 'preprocessed_text'], inplace=True)
df['views'] = pd.to_numeric(df['views'], errors='coerce') # Ensure views is numeric
print(f"DataFrame after dropping rows with missing essential metadata and coercing views: {len(df)} rows.")

# Load the fine-tuned NER model and tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(FINE_TUNED_MODEL_NAME_OR_PATH)
    model = AutoModelForTokenClassification.from_pretrained(FINE_TUNED_MODEL_NAME_OR_PATH)

    # Create an NER pipeline for efficient inference
    ner_pipeline = pipeline(
        "token-classification",
        model=model,
        tokenizer=tokenizer,
        aggregation_strategy="simple" # Aggregates sub-word tokens to full words
    )
    print("Fine-tuned NER model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error: Failed to load fine-tuned model or create pipeline from '{FINE_TUNED_MODEL_NAME_OR_PATH}': {e}")
    print("Please verify the model path/name and ensure it exists locally or on Hugging Face Hub.")
    raise

Loaded 2100 rows from /content/data/processed/preprocessed_telegram_data.csv

DataFrame Head after initial load:


,channel_title,channel_username,message_id,message_text,message_date,sender_id,sender_username,media_path,media_type,views,preprocessed_text,tokens
0,NEVA COMPUTER®,@nevacomputer,8779,💻 Dell Precision Workstation\n\nA powerful per...,2025-06-21T06:32:27+00:00,-1001195361398,nevacomputer,NaN,photo_skipped_download,835.0,Dell Precision Workstation A powerful performa...,"['Dell', 'Precision', 'Workstation', 'A', 'pow..."
1,NEVA COMPUTER®,@nevacomputer,8778,NaN,2025-06-21T06:32:27+00:00,-1001195361398,nevacomputer,NaN,photo_skipped_download,832.0,NaN,[]
2,NEVA COMPUTER®,@nevacomputer,8777,NaN,2025-06-21T06:32:27+00:00,-1001195361398,nevacomputer,NaN,photo_skipped_download,833.0,NaN,[]
3,NEVA COMPUTER®,@nevacomputer,8776,NaN,2025-06-21T06:32:27+00:00,-1001195361398,nevacomputer,NaN,photo_skipped_download,720.0,NaN,[]
4,NEVA COMPUTER®,@nevacomputer,8775,NaN,2025-06-11T13:56:52+00:00,-1001195361398,nevacomputer,NaN,photo_skipped_download,1788.0,NaN,[]



DataFrame Columns after initial load:
Index(['channel_title', 'channel_username', 'message_id', 'message_text',
       'message_date', 'sender_id', 'sender_username', 'media_path',
       'media_type', 'views', 'preprocessed_text', 'tokens'],
      dtype='object')
DataFrame after dropping rows with missing essential metadata and coercing views: 1203 rows.


Device set to use cpu


Fine-tuned NER model and tokenizer loaded successfully.


## 4. Perform NER Inference on All Data
This section utilizes the loaded NER pipeline to process the 'preprocessed_text' column of the DataFrame. For each message, the pipeline identifies and extracts entities based on the model's training. The results, which are lists of dictionaries containing information about the extracted entities (e.g., entity type, word, score), are stored in a new column called 'extracted_entities'. This process enriches the dataset with structured information derived from the unstructured text.

In [18]:
from tqdm.auto import tqdm

print("Starting NER inference on all preprocessed messages...")

# Apply NER to all preprocessed text
# This might take a while depending on the number of messages and GPU availability
ner_results = []
# Use tqdm to show progress
for i, text in tqdm(enumerate(df['preprocessed_text']), total=len(df), desc="Performing NER"):
    if pd.isna(text) or not text.strip():
        ner_results.append([]) # Append empty list for empty/NaN texts
        continue
    try:
        results = ner_pipeline(text)
        ner_results.append(results)
    except Exception as e:
        print(f"Warning during NER for message {i}: {e}. Appending empty result.")
        ner_results.append([])

df['extracted_entities'] = ner_results
print("NER inference complete. Extracted entities column added.")

Starting NER inference on all preprocessed messages...


Performing NER:   0%|          | 0/1203 [00:00<?, ?it/s]

NER inference complete. Extracted entities column added.


## 5. Entity Extraction and Normalization Helpers

This section provides helper functions designed to further process the raw NER output.
- `extract_entities_by_type`: This function filters the list of extracted entities for a given message and returns only the words corresponding to a specified entity type (e.g., 'PRODUCT', 'PRICE').
- `extract_numerical_price`: This function takes a list of tokens identified as a price and attempts to convert it into a numerical float value. It includes logic to handle common currency symbols, commas, and spaces found in the price strings, and gracefully handles cases where conversion is not possible by returning `np.nan`.

These helper functions are then applied to the 'extracted_entities' column to create dedicated columns for 'products', 'prices', 'locations', and 'contact_info'. The `extract_numerical_price` function is specifically used to create a 'numerical_prices' column, which is then flattened into 'all_numerical_prices' for easier aggregation.

In [19]:
from typing import List, Dict

def extract_entities_by_type(entities: List[Dict], entity_type: str) -> List[str]:
    """Extracts all entities of a specific type from a list of NER results."""
    return [ent['word'] for ent in entities if ent['entity_group'] == entity_type]

def extract_numerical_price(price_tokens: List[str]) -> float:
    """
    Extracts and converts a price entity (e.g., ['5000', 'ብር']) into a float.
    Handles commas, currency symbols, and ensures valid conversion.
    """
    if not price_tokens:
        return np.nan

    # Join tokens and convert to lowercase for robust matching
    full_price_str = "".join(price_tokens).lower()

    # Remove common Amharic currency terms and symbols and symbols, as well as commas and spaces
    # Add more variations as observed in data (e.g., 'ብር', ' birr', 'eth', 'etb', '$')
    price_value_str = re.sub(r'[ብርbirr\s,]', '', full_price_str)

    try:
        return float(price_value_str)
    except ValueError:
        print(f"WARNING: Could not convert price '{full_price_str}' to float. Returning NaN.")
        return np.nan

# Extract specific entity types into new columns
if 'extracted_entities' in df.columns:
    df['products'] = df['extracted_entities'].apply(lambda x: extract_entities_by_type(x, 'PRODUCT'))
    df['prices'] = df['extracted_entities'].apply(lambda x: extract_entities_by_type(x, 'PRICE'))
    df['locations'] = df['extracted_entities'].apply(lambda x: extract_entities_by_type(x, 'LOC'))
    df['contact_info'] = df['extracted_entities'].apply(lambda x: extract_entities_by_type(x, 'CONTACT_INFO'))

    # Convert extracted price strings to numerical values
    # Note: For multiple prices in one message, this will create a list of numerical prices.
    df['numerical_prices'] = df['prices'].apply(lambda x: [extract_numerical_price([p]) for p in x if p])
    # Flatten the list of numerical prices for easier average calculation across all products in a message
    df['all_numerical_prices'] = df['numerical_prices'].apply(lambda x: [val for val in x if not pd.isna(val)])

    print("Extracted specific entities and converted prices.")
else:
    print("Error: The 'extracted_entities' column was not found in the DataFrame. Please ensure the previous step (NER inference) was executed successfully.")

Extracted specific entities and converted prices.


## 6. Develop Vendor Analytics Engine & Calculate Key Metrics

This section defines the core logic for calculating vendor-specific performance metrics. The `calculate_vendor_metrics` function takes a DataFrame filtered for a single vendor and computes the following:
- **Total Posts:** The total number of messages posted by the vendor.
- **Average Views per Post:** The average view count across all of the vendor's posts, indicating content reach.
- **Posting Frequency (Posts/Week):** The number of posts divided by the duration of the vendor's activity in weeks, showing consistency and activity level.
- **Average Price Point (ETB):** The average of all numerical prices extracted from the vendor's posts, providing insight into their typical product pricing.
- **Top Performing Post (Product & Price):** Identifies the post with the highest view count and extracts the product and price associated with that post.

This function is designed to provide a comprehensive snapshot of a vendor's performance based on the available data.

In [20]:
def calculate_vendor_metrics(df_vendor: pd.DataFrame) -> Dict:
    """
    Calculates key performance metrics for a single vendor.

    Args:
        df_vendor (pd.DataFrame): DataFrame containing posts for a single vendor.

    Returns:
        Dict: A dictionary of calculated metrics for the vendor.
    """
    if df_vendor.empty:
        return {
            'Avg. Views/Post': 0,
            'Posts/Week': 0,
            'Avg. Price (ETB)': 0,
            'Top Product': 'N/A',
            'Top Product Price': 'N/A',
            'Total Posts': 0,
            'message_date Range Days': 0
        }

    # Total Posts
    total_posts = len(df_vendor)

    # Average Views per Post
    avg_views_per_post = df_vendor['views'].mean() if not df_vendor['views'].dropna().empty else 0

    # Posting Frequency (Posts per Week)
    min_message_date = df_vendor['message_date'].min()
    max_message_date = df_vendor['message_date'].max()
    message_date_range_days = (max_message_date - min_message_date).days + 1 # +1 to include both start and end day

    if message_date_range_days <= 0: # Handle cases where there's only one post or invalid message_date range
        posting_frequency = total_posts # Consider all posts within a "single period"
        message_date_range_days = 1 # Set to 1 to avoid division by zero later if used
    else:
        posting_frequency = total_posts / (message_date_range_days / 7) # Posts per 7 days

    # Average Price Point
    # Flatten all prices from all posts of the vendor
    all_prices_flat = [price for sublist in df_vendor['all_numerical_prices'] for price in sublist]
    avg_price_point = np.mean(all_prices_flat) if all_prices_flat else np.nan

    # Top Performing Post - find the message with the highest view count
    top_post = df_vendor.loc[df_vendor['views'].idxmax()]
    # Get the first product if available, otherwise 'N/A'
    top_product = top_post['products'][0] if top_post['products'] else 'N/A'
    # Get the first numerical price if available, otherwise NaN
    top_product_price = top_post['all_numerical_prices'][0] if top_post['all_numerical_prices'] else np.nan

    return {
        'Total Posts': total_posts,
        'Avg. Views/Post': avg_views_per_post,
        'Posts/Week': posting_frequency,
        'Avg. Price (ETB)': avg_price_point,
        'Top Product': top_product,
        'Top Product Price': top_product_price,
        'message_date Range Days': message_date_range_days # Useful for debugging frequency
    }

## 7. Process All Vendors

This section orchestrates the application of the `calculate_vendor_metrics` function to every unique vendor in the dataset. It iterates through the list of unique channel usernames, filters the main DataFrame to get the posts for each vendor, and then calls the metrics calculation function. The calculated metrics for each vendor are collected in a list, which is then converted into a new pandas DataFrame called `vendor_scorecard_df`. This DataFrame serves as the foundation for generating the final vendor scorecard.

In [21]:
from tqdm.auto import tqdm

print("Calculating metrics for all unique vendors...")
vendor_metrics_list = []
unique_vendors = df['channel_username'].unique()

for vendor_name in unique_vendors:
    # Ensure 'all_numerical_prices' is included when creating the vendor-specific DataFrame
    df_vendor = df[df['channel_username'] == vendor_name].copy()
    metrics = calculate_vendor_metrics(df_vendor)
    metrics['Vendor'] = vendor_name
    vendor_metrics_list.append(metrics)

vendor_scorecard_df = pd.DataFrame(vendor_metrics_list)

print("Vendor metrics calculated.")
print("\nRaw Vendor Metrics:")
# Check if the DataFrame is empty before trying to print columns
if not vendor_scorecard_df.empty:
    print(vendor_scorecard_df.columns)
    print(vendor_scorecard_df.head())
    print(vendor_scorecard_df[['Vendor', 'Total Posts', 'Avg. Views/Post', 'Posts/Week', 'Avg. Price (ETB)', 'Top Product', 'Top Product Price']].to_string())
else:
    print("No vendor metrics were calculated. The vendor scorecard DataFrame is empty.")

Calculating metrics for all unique vendors...
Vendor metrics calculated.

Raw Vendor Metrics:
Index(['Total Posts', 'Avg. Views/Post', 'Posts/Week', 'Avg. Price (ETB)',
       'Top Product', 'Top Product Price', 'message_date Range Days',
       'Vendor'],
      dtype='object')
   Total Posts  Avg. Views/Post  Posts/Week  Avg. Price (ETB)  \
0           32      3507.093750    0.785965      3.557162e+03   
1           87      4547.379310    4.229167      8.805000e+02   
2           99     28728.505051    7.786517      1.896103e+03   
3           55     15902.327273   17.500000      2.536605e+09   
4           25      3220.960000   17.500000      1.815667e+03   

           Top Product  Top Product Price  message_date Range Days  \
0    : DELL G15 : 15.6                4.0                      285   
1            swing car              120.0                      144   
2   Nike infinity flow             3500.0                       89   
3  Sokany Meat Grinder                3.0         

## Interpretation of Key Vendor Metrics
**Average Views per Post:**
This metric directly reflects a vendor's market reach and the level of customer engagement their content generates. A higher average view count indicates that the vendor's posts are being seen by more potential customers, suggesting broader visibility and potentially more interest in their products.

**Posting Frequency (Posts per Week):**
This metric indicates a vendor's activity and consistency. A higher posting frequency suggests an active and potentially reliable business that regularly engages its audience and upmessage_dates its product offerings. Consistent activity is a positive signal for potential micro-lending.

**Average Price Point (ETB):**
This metric provides insight into a vendor's business profile and market segment. A higher average price point might indicate a vendor dealing in higher-value, lower-volume products (e.g., electronics, specialized machinery), while a lower average price point could suggest a high-volume, lower-margin business (e.g., common household goods, fast-moving consumer goods). This helps lenders understand the scale and nature of the business.

**Top Performing Post (Product & Price):**
Identifying the product and its price from the highest-viewed post helps understand what kind of content or product resonates most with a vendor's audience. This provides qualitative insight into their most successful offerings, which can be valuable for business assessment.

## 8. Create a Final "Lending Score"

To create a single, comparable metric for ranking vendors, a "Lending Score" is calculated. This section first normalizes the key numerical metrics ('Avg. Views/Post', 'Posts/Week', 'Avg. Price (ETB)') using Min-Max scaling. Normalization is essential because it brings the different metrics onto a similar scale (0 to 1), preventing metrics with larger absolute values from dominating the final score.

A dictionary `METRIC_WEIGHTS` assigns importance to each normalized metric. These weights can be adjusted based on which factors are considered most critical for micro-lending decisions. The Lending Score is then computed as a weighted sum of the normalized metrics. Finally, the score is optionally scaled to a 0-100 range for easier interpretation, providing a clear indication of a vendor's potential based on their Telegram activity and extracted business details.

In [22]:
# Normalize metrics before combining them to ensure fair weighting
# We will use Min-Max scaling: (X - min(X)) / (max(X) - min(X))

# Define metrics to normalize and their importance (weights)
# Example weights, these can be adjusted based on business priorities
# Views and Posting Frequency are often good indicators of engagement.
# Price point might indicate market segment (luxury vs. mass market).
METRIC_WEIGHTS = {
    'Avg. Views/Post': 0.4,
    'Posts/Week': 0.4,
    'Avg. Price (ETB)': 0.2, # Lower weight as it defines market segment, not directly engagement
}

# Ensure columns exist and handle NaNs for normalization
for col in METRIC_WEIGHTS.keys():
    if col not in vendor_scorecard_df.columns:
        vendor_scorecard_df[col] = 0.0 # Add column with zeros if missing
    # Replace NaNs with 0 for metrics where it makes sense (e.g., if no prices were extracted)
    vendor_scorecard_df[col] = vendor_scorecard_df[col].fillna(0)

# Normalize relevant columns
normalized_df = vendor_scorecard_df.copy()
for metric, weight in METRIC_WEIGHTS.items():
    min_val = normalized_df[metric].min()
    max_val = normalized_df[metric].max()

    if max_val == min_val: # Avoid division by zero if all values are the same
        normalized_df[f'Normalized {metric}'] = 0.0
    else:
        normalized_df[f'Normalized {metric}'] = (normalized_df[metric] - min_val) / (max_val - min_val)

    print(f"Normalized '{metric}' (Min: {min_val:.2f}, Max: {max_val:.2f})")

# Calculate the Lending Score
normalized_df['Lending Score'] = 0.0
for metric, weight in METRIC_WEIGHTS.items():
    normalized_df['Lending Score'] += normalized_df[f'Normalized {metric}'] * weight

# Optionally scale Lending Score to 0-100 or another range for easier interpretation
max_possible_score = sum(METRIC_WEIGHTS.values()) # Should be 1.0 if weights sum to 1
normalized_df['Lending Score (0-100)'] = (normalized_df['Lending Score'] / max_possible_score) * 100

print("Lending Score calculated.")

Normalized 'Avg. Views/Post' (Min: 1477.43, Max: 28728.51)
Normalized 'Posts/Week' (Min: 0.79, Max: 28.00)
Normalized 'Avg. Price (ETB)' (Min: 3.15, Max: 2536605318.97)
Lending Score calculated.


## Interpretation of the Lending Score
The 'Lending Score' is a composite metric designed to identify promising vendors for micro-lending. It is calculated as a weighted sum of normalized key performance indicators:
  - Average Views per Post (Weight: 0.4): Reflects market reach and customer engagement.
  - Posting Frequency (Weight: 0.4): Indicates business activity and consistency.
  - Average Price Point (Weight: 0.2): Provides insight into the vendor's market segment (e.g., high-value vs. high-volume goods).

A higher Lending Score (on a scale of 0-100) suggests a vendor with stronger engagement, consistent activity, and a potentially viable business model based on their Telegram presence. This score serves as a data-driven input for EthioMart's micro-lending decisions, complementing traditional financial assessments.

## 9. Present Vendor Scorecard Table

This section generates and displays the final vendor scorecard in a clear, tabular format. It selects the most relevant columns from the processed data ('Vendor', 'Avg. Views/Post', 'Posts/Week', 'Avg. Price (ETB)', 'Top Product', 'Top Product Price', 'Lending Score (0-100)') and sorts the table in descending order based on the 'Lending Score (0-100)'. The `to_string(index=False)` method is used to display the entire table without the pandas DataFrame index, making it easy to read and interpret the rankings and performance indicators for each vendor.

In [23]:
final_scorecard_display = normalized_df[[
    'Vendor',
    'Avg. Views/Post',
    'Posts/Week',
    'Avg. Price (ETB)',
    'Top Product',
    'Top Product Price',
    'Lending Score (0-100)'
]].sort_values(by='Lending Score (0-100)', ascending=False) # Sort by lending score

print("\n--- FinTech Vendor Scorecard Summary ---")
print("The table below presents the calculated metrics and the composite 'Lending Score' for each analyzed vendor. Vendors are sorted by their Lending Score in descending order, with higher scores indicating potentially more promising candimessage_dates for micro-lending.")
print(final_scorecard_display.to_string(index=False))


--- FinTech Vendor Scorecard Summary ---
The table below presents the calculated metrics and the composite 'Lending Score' for each analyzed vendor. Vendors are sorted by their Lending Score in descending order, with higher scores indicating potentially more promising candimessage_dates for micro-lending.
                 Vendor  Avg. Views/Post  Posts/Week  Avg. Price (ETB)               Top Product  Top Product Price  Lending Score (0-100)
               @Leyueqa     15902.327273   17.500000      2.536605e+09       Sokany Meat Grinder                3.0              65.740110
              @MerttEka     12732.729167   28.000000      7.122320e+02                         W                6.0              56.520890
@ethio_brand_collection     28728.505051    7.786517      1.896103e+03        Nike infinity flow             3500.0              50.289635
              @qnashcom      9688.548780   20.140351      9.409569e+02             Spa Gel Socks                NaN              40.5002

## Result Summary and Key Insights

This section provides a concise summary of the analysis results, focusing on the vendors with the highest and lowest Lending Scores.

**Top Performing Vendor:** The vendor with the highest Lending Score is highlighted, along with their key metrics. This vendor demonstrates strong online activity (high posting frequency), significant market reach (high average views), and potentially a viable business model based on their price point and top-performing product. This indicates they are a promising candidate for micro-lending based on their digital footprint.

**Lowest Performing Vendor:** For comparison, the vendor with the lowest Lending Score is also presented with their metrics. This vendor exhibits lower activity and engagement levels based on the Telegram data, suggesting a less compelling profile for micro-lending when considering only their online presence.

**Key Insights:** The summary emphasizes that the Lending Score is a data-driven tool to complement traditional financial assessments. It allows EthioMart to identify potential micro-lending recipients by quantifying their online business health and engagement. The top-ranked vendors represent those who are most actively leveraging their Telegram channels and successfully reaching their audience, making them potentially lower-risk and higher-opportunity candidates for micro-lending. The scorecard provides a quick and interpretable ranking to guide initial lending considerations.

In [24]:
if not final_scorecard_display.empty:
    top_vendor = final_scorecard_display.iloc[0]
    print("\n**Top Performing Vendor:**")
    print(f"  - Vendor: {top_vendor['Vendor']}")
    print(f"  - Lending Score (0-100): {top_vendor['Lending Score (0-100)']:.2f}")
    print(f"  - Avg. Views/Post: {top_vendor['Avg. Views/Post']:.2f}")
    print(f"  - Posts/Week: {top_vendor['Posts/Week']:.2f}")
    print(f"  - Average Price (ETB): {top_vendor['Avg. Price (ETB)'] if not pd.isna(top_vendor['Avg. Price (ETB)']) else 'N/A'}")
    print(f"  - Top Product: {top_vendor['Top Product']}")
    print(f"  - Top Product Price: {top_vendor['Top Product Price'] if not pd.isna(top_vendor['Top Product Price']) else 'N/A'}")
    print("\nThis vendor demonstrates strong online activity and customer engagement, making them a prime candimessage_date for micro-lending based on their Telegram presence.")

    if len(final_scorecard_display) > 1:
        bottom_vendor = final_scorecard_display.iloc[-1]
        print("\n**Lowest Performing Vendor (for comparison):**")
        print(f"  - Vendor: {bottom_vendor['Vendor']}")
        print(f"  - Lending Score (0-100): {bottom_vendor['Lending Score (0-100)']:.2f}")
        print(f"  - Avg. Views/Post: {bottom_vendor['Avg. Views/Post']:.2f}")
        print(f"  - Posts/Week: {bottom_vendor['Posts/Week']:.2f}")
        print(f"  - Average Price (ETB): {bottom_vendor['Avg. Price (ETB)'] if not pd.isna(bottom_vendor['Avg. Price (ETB)']) else 'N/A'}")
        print(f"  - Top Product: {bottom_vendor['Top Product']}")
        print(f"  - Top Product Price: {bottom_vendor['Top Product Price'] if not pd.isna(bottom_vendor['Top Product Price']) else 'N/A'}")
        print("\nThis vendor shows lower activity and engagement, indicating a less promising profile for micro-lending based solely on Telegram data.")
else:
    print("No vendor data available to generate a summary.")



**Top Performing Vendor:**
  - Vendor: @Leyueqa
  - Lending Score (0-100): 65.74
  - Avg. Views/Post: 15902.33
  - Posts/Week: 17.50
  - Average Price (ETB): 2536605318.968421
  - Top Product: Sokany Meat Grinder
  - Top Product Price: 3.0

This vendor demonstrates strong online activity and customer engagement, making them a prime candimessage_date for micro-lending based on their Telegram presence.

**Lowest Performing Vendor (for comparison):**
  - Vendor: @aradabrand2
  - Lending Score (0-100): 1.76
  - Avg. Views/Post: 2516.27
  - Posts/Week: 0.95
  - Average Price (ETB): 36.75
  - Top Product: High Standard Clothing
  - Top Product Price: 36.0

This vendor shows lower activity and engagement, indicating a less promising profile for micro-lending based solely on Telegram data.

Vendor scorecard saved to: /content/data/processed/vendor_scorecard.csv


## 10. Save the Scorecard

The final step is to save the generated vendor scorecard DataFrame to a CSV file at the path specified in the configuration section (`VENDOR_SCORECARD_PATH`). This ensures that the results of the analysis are persisted and can be easily accessed for further analysis, reporting, or integration into other systems. The CSV is saved without the DataFrame index and with UTF-8 encoding to handle potential special characters in vendor names or product descriptions.

In [ ]:
# Save the Scorecard
os.makedirs(VENDOR_SCORECARD_PATH.parent, exist_ok=True)
final_scorecard_display.to_csv(VENDOR_SCORECARD_PATH, index=False, encoding='utf-8')
print(f"\nVendor scorecard saved to: {VENDOR_SCORECARD_PATH}")